In [42]:
import pandas as pd
import math
from decimal import *
import numpy as np

In [43]:
dfAden = pd.read_csv('../data/aden.csv',sep=';')

# Ajustement

In [44]:
dfAden['Annee (0)'] = dfAden['Annee (0)'].astype(int)
dfAden['jour (0)'] = dfAden['jour (0)'].astype(int)
dfAden['mois (0)'] = dfAden['mois (0)'].astype(int)

dfAden['descriptif']=dfAden.apply(lambda row: row['descriptif'].replace('\r', ''), axis=1)

# Affectation grandes zones

Dans cette section, on définit les grandes zones :
- Golfe d'aden
- Est
- Sud

In [4]:
golfe_min_lat=10
golfe_max_lat=17
golfe_min_lon=40
golfe_max_lon=56

arabie_min_lon=56
golfe_min_lat=10

def choose_large_area(latitude,longitude):
    if latitude<golfe_max_lat and latitude>golfe_min_lat and longitude<=golfe_max_lon and longitude>golfe_min_lon :
        return"G"
    elif latitude>golfe_min_lat and longitude>arabie_min_lon:
        return"A"
    else:
        return"I"
    
#print choose_large_area(11,41)

dfAden['large_area'] = dfAden.apply(lambda row: choose_large_area(row['latitude'],row['longitude']), axis=1)

dfAden.to_csv("../data/aden.csv",sep=';', index=False)

# Affectation des sous zones

In [7]:
dfAden = pd.read_csv('../data/aden.csv',sep=';')
print dfAden.columns

Index([u'Annee (0)', u'jour (0)', u'mois (0)', u'dist �� la c��te (nq)',
       u'zone', u'lat (0)', u'long(0)', u'type(0)', u'pavillon',
       u'statut de l'attaque(0)', u'type d'attaque(0)', u'Nombre P  Min',
       u'Nombre P  max', u'nombre embarcation', u'Vol �� bord', u' Kidnapping',
       u'bless��es', u' meurtre', u'Knives ', u'RPG', u'descriptif',
       u'latitude', u'longitude', u'quinzaine_week', u'newDescriptif', u'Guns',
       u'mousson', u'distance au port', u'new_type', u'large_area'],
      dtype='object')


## Sous grille automatique

Cette fonction renvoie deux tableaux contenant le découpage en zones

In [17]:
def get_grid_area(min_lat,max_lat,min_lon,max_lon,accuracy_km=100): 
    #accuracy_km est la précision que l'on veut donner
    #un degre de latitude "vaut"  111,11 km
    accuracy_lat=accuracy_km/111.11

    diff_lat=max_lat-min_lat
    diff_lon=max_lon-min_lon

    path_lat=float(diff_lat)/float(accuracy_lat)

    latitude_grid=np.linspace(min_lat, max_lat, path_lat)
    longitude_grid=[]
    longitude_grid.append(golfe_min_lon)

    tab=[]
    for i in range(0,len(latitude_grid)-1):
        mean_lat=float(latitude_grid[i]+latitude_grid[i+1])/2
        km_lon=111.11*math.cos(math.radians(mean_lat))
        accuracy_lon=accuracy_km/km_lon
        tab.append(accuracy_lon) 
        currentLon=longitude_grid[i]+accuracy_lon
        longitude_grid.append(currentLon)

    #on complete jusqu'a golfe_max_lon
    maxCurrent=max(longitude_grid)
    meanPath=np.mean(tab)
    while maxCurrent<max_lon:
        maxCurrent=max(longitude_grid)
        longitude_grid.append(maxCurrent+meanPath)
    
    return latitude_grid.tolist(),longitude_grid


golfe_latitude_grid,golfe_longitude_grid=get_grid_area(golfe_min_lat,golfe_max_lat,golfe_min_lon,golfe_max_lon)


print len(golfe_latitude_grid)
print len(golfe_longitude_grid)
print golfe_latitude_grid
print golfe_longitude_grid

7
20
[10.0, 11.166666666666666, 12.333333333333334, 13.5, 14.666666666666668, 15.833333333333334, 17.0]
[40, 40.91558417010453, 41.834856078697285, 42.758229934975276, 43.6861289450159, 44.618986350851486, 45.55724652549592, 46.483454279745239, 47.409662033994557, 48.335869788243876, 49.262077542493195, 50.188285296742514, 51.114493050991832, 52.040700805241151, 52.96690855949047, 53.893116313739789, 54.819324067989108, 55.745531822238426, 56.671739576487745, 57.597947330737064]


On utilise ce découpage pour créer deux colonnes : lat_area et lon_area

In [18]:
def affectation_zone(latitude,longitude,latitude_grid,longitude_grid,lat_bool):
    #print 'latitude : '+ str(latitude)
    #print 'longitude : '+ str(longitude)
    #print latitude_grid
    idx_zone_lat=None
    idx_zone_lon=None
    
    for i in range(0,len(latitude_grid)-1):        
        if latitude<=latitude_grid[i+1] and latitude>latitude_grid[i]:
            idx_zone_lat=i

    for i in range(0,len(longitude_grid)-1):
        if longitude<=longitude_grid[i+1] and longitude>longitude_grid[i]:
            idx_zone_lon=i

    #print idx_zone_lon
    #print idx_zone_lat
    if lat_bool:
        return idx_zone_lat
    else:
        return idx_zone_lon

In [19]:
dfAden['lat_area'] = dfAden[dfAden['large_area']=="G"].apply(lambda row: affectation_zone(row['latitude'],row['longitude'],golfe_latitude_grid,golfe_longitude_grid,True), axis=1)
dfAden['lon_area'] = dfAden[dfAden['large_area']=="G"].apply(lambda row: affectation_zone(row['latitude'],row['longitude'],golfe_latitude_grid,golfe_longitude_grid,False), axis=1)

In [20]:
dfAden[dfAden['large_area']=="G"].head(3)

,Annee (0),jour (0),mois (0),dist cote (nq),zone,lat (0),long(0),type(0),pavillon,statut de l'attaque(0),...,newDescriptif,Guns,mousson,distance au port,distance_shorline,lat_area,lon_area,large_area,distance a la base de pirate,id_cluster
0,2010,28,10,NaN,BCD,12:08N,054:25E,Dhow,Not Stated,Steaming,...,pirates hijacked a dhow.,0,0,534.679104,35.872306,1,15,G,346.662210,2010-10_2
1,2010,30,10,NaN,BCD,15:06N,055:58E,Bulk Carrier,Malta,Steaming,...,armed pirates attacked and boarded the ship un...,0,0,292.330113,233.517911,4,17,G,626.662668,2010-10_2
2,2010,1,1,NaN,BCD,12:31N,047:18E,Chemical Tanker,Singapore,Steaming,...,pirates hijacked a tanker underway and took he...,0,0,259.217741,118.411436,2,7,G,244.881989,2010-01_1


## Sous grille manuelle

On définit les zones manuellement.

In [23]:
global zone_A_min_lat
zone_A_min_lat=14.842
global zone_A_max_lat
zone_A_max_lat=16.049
global zone_A_min_lon
zone_A_min_lon=41.22
global zone_A_max_lon
zone_A_max_lon=42.47

zone_B_min_lat=13.928
zone_B_max_lat=14.835
zone_B_min_lon=41.83
zone_B_max_lon=43.06

zone_C_min_lat=13.081
zone_C_max_lat=13.930
zone_C_min_lon=42.26
zone_C_max_lon=43.29

zone_D_min_lat=12.531
zone_D_max_lat=zone_C_min_lat
zone_D_min_lon=43.08
zone_D_max_lon=43.50

zone_E_min_lat=12.049
zone_E_max_lat=zone_D_min_lat
zone_E_min_lon=zone_D_max_lon
zone_E_max_lon=44.47

zone_F_min_lat=11.737
zone_F_max_lat=12.931
zone_F_min_lon=zone_E_max_lon
zone_F_max_lon=45.77

zone_G_min_lat=zone_F_min_lat
zone_G_max_lat=13.386
zone_G_min_lon=45.76
zone_G_max_lon=46.84

zone_H_min_lat=zone_F_min_lat
zone_H_max_lat=13.397
zone_H_min_lon=zone_G_max_lon
zone_H_max_lon=47.84

zone_I_min_lat=12.340
zone_I_max_lat=14.588
zone_I_min_lon=zone_H_min_lon
zone_I_max_lon=49.36

zone_J_min_lat=zone_I_min_lat
zone_J_max_lat=zone_I_max_lat
zone_J_min_lon=zone_I_max_lon
zone_J_max_lon=50.58

zone_K_min_lat=zone_I_min_lat
zone_K_max_lat=15.819
zone_K_min_lon=zone_J_max_lon
zone_K_max_lon=52.27

zone_L_min_lat=zone_I_min_lat
zone_L_max_lat=16.761
zone_L_min_lon=52.27
zone_L_max_lon=56.13

zone_M_min_lat=10.450
zone_M_max_lat=zone_I_min_lat
zone_M_min_lon=zone_H_max_lon
zone_M_max_lon=55.01


In [41]:
def get_area_by_hand(lat,lon): 
    zone=""
    if zone_A_max_lat>lat>zone_A_min_lat and zone_A_max_lon>lon>zone_A_min_lon:
        zone="A"
    elif zone_B_max_lat>lat>zone_B_min_lat and zone_B_max_lon>lon>zone_B_min_lon:
        zone="B"
    elif zone_C_max_lat>lat>zone_C_min_lat and zone_C_max_lon>lon>zone_C_min_lon:
        zone="C"
    elif zone_D_max_lat>lat>zone_D_min_lat and zone_D_max_lon>lon>zone_D_min_lon:
        zone="D"
    elif zone_E_max_lat>lat>zone_E_min_lat and zone_E_max_lon>lon>zone_E_min_lon:
        zone="E"
    elif zone_F_max_lat>lat>zone_F_min_lat and zone_F_max_lon>lon>zone_F_min_lon:
        zone="F"
    elif zone_G_max_lat>lat>zone_G_min_lat and zone_G_max_lon>lon>zone_G_min_lon:
        zone="G"
    elif zone_H_max_lat>lat>zone_H_min_lat and zone_H_max_lon>lon>zone_H_min_lon:
        zone="H"
    elif zone_I_max_lat>lat>zone_I_min_lat and zone_I_max_lon>lon>zone_I_min_lon:
        zone="I"
    elif zone_J_max_lat>lat>zone_J_min_lat and zone_J_max_lon>lon>zone_J_min_lon:
        zone="J"
    elif zone_K_max_lat>lat>zone_K_min_lat and zone_K_max_lon>lon>zone_K_min_lon:
        zone="K"
    elif zone_L_max_lat>lat>zone_L_min_lat and zone_L_max_lon>lon>zone_L_min_lon:
        zone="L"
    elif zone_M_max_lat>lat>zone_M_min_lat and zone_M_max_lon>lon>zone_M_min_lon:
        zone="M"
    return zone

print get_area_by_hand(float(15.0),float(41.5))
print get_area_by_hand(float(14.7),float(41.90))
print get_area_by_hand(float(13.5),float(42.3))
print get_area_by_hand(float(13.05),float(43.2))
print get_area_by_hand(float(12.5),float(43.6))
print get_area_by_hand(float(11.8),float(44.6))
print get_area_by_hand(float(11.8),float(45.9))
print get_area_by_hand(float(11.8),float(47))
print get_area_by_hand(float(14.5),float(48))
print get_area_by_hand(float(14.5),float(50))
print get_area_by_hand(float(14.5),float(51))
print get_area_by_hand(float(12),float(53))


A
B
C
D
E
F
G
H
I
J
K
M


In [47]:
dfAden['area_by_hand'] = dfAden.apply(lambda row: get_area_by_hand(float(row['latitude']),float(row['longitude'])), axis=1)
print dfAden['area_by_hand']

0       
1       
2       
3       
4       
5       
6       
7       
8       
9       
10      
11      
12      
13      
14      
15      
16      
17      
18      
19      
20      
21      
22      
23     M
24      
25     L
26      
27      
28      
29      
      ..
879     
880    J
881    K
882    E
883    C
884    C
885    H
886    I
887     
888     
889     
890     
891     
892     
893     
894     
895     
896    L
897     
898    L
899    E
900    L
901    C
902    D
903    E
904    E
905    D
906    A
907     
908     
Name: area_by_hand, dtype: object


## On ecrit un fichier pour la zone golfe

In [48]:
dfAden[dfAden['large_area']=="G"].to_csv("../data/aden_golfe.csv",sep=';', index=False)